## Desafio 01. Média móvel para transações bancárias

### Bibliotecas

In [1]:
import pandas as pd

# Módulo de conexão com banco
import sys
sys.path.append('../') 
import funcoesconexao as con

# Desativa warnings
import warnings
warnings.filterwarnings('ignore')

### Carregamento dos Dados

#### Via banco de dados — SQL Server

In [2]:
conexao = con.conectar_bd()

----- Conexão com banco de dados estabelecida com sucesso! -----


In [3]:
# Carregando dataframes
df_transacoes_bancarias = con.carregar_dados(conexao, 'transacoes_bancarias')

# Fechando conexão
con.fechar_conexao_bd(conexao)

In [4]:
# Exibindo dataframes
display(df_transacoes_bancarias.head())

,id,data_criacao,valor_transacao
0,1,2020-10-01 13:15:45,100.00
1,2,2020-10-01 15:10:00,55.90
2,3,2020-10-02 08:15:30,12.50
3,4,2020-10-02 08:35:00,220.00
4,5,2020-10-02 09:40:50,330.88


#### Via arquivo .CSV

In [5]:
# Carregando dataframes
df_transacoes_bancarias = con.carregar_dados_csv('transacoes_bancarias.csv')

In [6]:
# Exibindo dataframes
display(df_transacoes_bancarias.head())

,id,data_criacao,valor_transacao
0,1,2020-10-01 13:15:45.000,100
1,2,2020-10-01 15:10:00.000,"55,9"
2,3,2020-10-02 08:15:30.000,"12,5"
3,4,2020-10-02 08:35:00.000,220
4,5,2020-10-02 09:40:50.000,"330,88"


### Solução

In [7]:
def calcular_media_movel_depositos(transacoes_bancarias: pd.DataFrame):
    # Convertendo 'data_criacao' para datetime
    transacoes_bancarias['data_criacao'] = pd.to_datetime(
        transacoes_bancarias['data_criacao'], errors='coerce'
    )

    # Convertedo 'valor_transacao' para float
    transacoes_bancarias['valor_transacao'] = (
        transacoes_bancarias['valor_transacao'].str.replace(',', '.').astype(float)
    )

    # Criando coluna data_transacao
    transacoes_bancarias['data_transacao'] = pd.to_datetime(transacoes_bancarias['data_criacao'], format='%d-%m-%Y').dt.date

    # Filtrando apenas depósitos
    transacoes_bancarias = transacoes_bancarias[transacoes_bancarias.valor_transacao > 0]

    # Agrupando por data e calculando a somatória de cada dia
    transacoes_bancarias_agrupado = (
        transacoes_bancarias.groupby('data_transacao')
        .agg(valor_transacao=('valor_transacao', 'sum'))
        .reset_index()
    )

    # Calculando média móvel
    transacoes_bancarias_agrupado['media_movel'] = round(
        transacoes_bancarias_agrupado['valor_transacao'].rolling(3, min_periods=1).mean(),
        2,
    )

    # Removendo valores nulos
    resultado = transacoes_bancarias_agrupado.dropna()[['data_transacao', 'media_movel']]

    # Saída
    return resultado

In [8]:
calcular_media_movel_depositos(df_transacoes_bancarias)

,data_transacao,media_movel
0,2020-10-01,155.90
1,2020-10-02,359.64
2,2020-10-03,252.89
3,2020-10-04,400.93
4,2020-10-05,285.67
